# Machine Learning with H2O - Tutorial 3c: Regression Models (Ensembles)

<hr>

**Objective**:

- This tutorial explains how to create stacked ensembles of regression models for better out-of-bag performance.

<hr>

**Wine Quality Dataset:**

- Source: https://archive.ics.uci.edu/ml/datasets/Wine+Quality
- CSV (https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv)

<hr>
    
**Steps**:

1. Build GBM models using random grid search and extract the best one.
2. Build DRF models using random grid search and extract the best one. 
3. Build DNN models using random grid search and extract the best one.
4. Use model stacking to combining different models.


<hr>

**Full Technical Reference:**

- http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html
- http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/stacked-ensembles.html

<br>


In [1]:
# Import all required modules
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch

# Start and connect to a local H2O cluster
h2o.init(nthreads = -1)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,12 mins 47 secs
H2O cluster version:,3.10.4.3
H2O cluster version age:,5 days
H2O cluster name:,H2O_from_python_joe_bkd7hp
H2O cluster total nodes:,1
H2O cluster free memory:,4.835 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


<br>

In [2]:
# Import wine quality data from a local CSV file
wine = h2o.import_file("winequality-white.csv")
wine.head(5)

Parse progress: |█████████████████████████████████████████████████████████| 100%


fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6
6.3,0.3,0.34,1.6,0.049,14,132,0.994,3.3,0.49,9.5,6
8.1,0.28,0.4,6.9,0.05,30,97,0.9951,3.26,0.44,10.1,6
7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6
7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.4,9.9,6


In [3]:
# Define features (or predictors)
features = list(wine.columns) # we want to use all the information
features.remove('quality')    # we need to exclude the target 'quality' (otherwise there is nothing to predict)
features

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [4]:
# Split the H2O data frame into training/test sets
# so we can evaluate out-of-bag performance
wine_split = wine.split_frame(ratios = [0.8], seed = 1234)

wine_train = wine_split[0] # using 80% for training
wine_test = wine_split[1]  # using the rest 20% for out-of-bag evaluation

In [5]:
wine_train.shape

(3932, 12)

In [6]:
wine_test.shape

(966, 12)

<br>

## Define Search Criteria for Random Grid Search

In [7]:
# define the criteria for random grid search
search_criteria = {'strategy': "RandomDiscrete", 
                   'max_models': 9,
                   'seed': 1234}

<br>

## Step 1: Build GBM Models using Random Grid Search and Extract the Best Model

In [8]:
# define the range of hyper-parameters for GBM grid search
# 27 combinations in total
hyper_params = {'sample_rate': [0.7, 0.8, 0.9],
                'col_sample_rate': [0.7, 0.8, 0.9],
                'max_depth': [3, 5, 7]}

In [9]:
# Set up GBM grid search
# Add a seed for reproducibility
gbm_rand_grid = H2OGridSearch(
                    H2OGradientBoostingEstimator(
                        model_id = 'gbm_rand_grid', 
                        seed = 1234,
                        ntrees = 10000,   
                        nfolds = 5,
                        fold_assignment = "Modulo",               # needed for stacked ensembles
                        keep_cross_validation_predictions = True, # needed for stacked ensembles
                        stopping_metric = 'mse', 
                        stopping_rounds = 15,     
                        score_tree_interval = 1),
                    search_criteria = search_criteria, 
                    hyper_params = hyper_params)

In [10]:
# Use .train() to start the grid search
gbm_rand_grid.train(x = features, 
                    y = 'quality', 
                    training_frame = wine_train)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [11]:
# Sort and show the grid search results
gbm_rand_grid_sorted = gbm_rand_grid.get_grid(sort_by='mse', decreasing=False)
print(gbm_rand_grid_sorted)

    col_sample_rate max_depth sample_rate  \
0               0.9         7         0.9   
1               0.8         7         0.7   
2               0.7         7         0.7   
3               0.9         7         0.7   
4               0.7         5         0.8   
5               0.8         3         0.9   
6               0.7         3         0.7   
7               0.9         3         0.9   
8               0.8         3         0.8   

                                                     model_ids  \
0  Grid_GBM_py_4_sid_85d6_model_python_1491487273261_4_model_5   
1  Grid_GBM_py_4_sid_85d6_model_python_1491487273261_4_model_4   
2  Grid_GBM_py_4_sid_85d6_model_python_1491487273261_4_model_1   
3  Grid_GBM_py_4_sid_85d6_model_python_1491487273261_4_model_6   
4  Grid_GBM_py_4_sid_85d6_model_python_1491487273261_4_model_0   
5  Grid_GBM_py_4_sid_85d6_model_python_1491487273261_4_model_7   
6  Grid_GBM_py_4_sid_85d6_model_python_1491487273261_4_model_8   
7  Grid_GBM_py_4_sid_

In [12]:
# Extract the best model from random grid search
best_gbm_model_id = gbm_rand_grid_sorted.model_ids[0]
best_gbm_from_rand_grid = h2o.get_model(best_gbm_model_id)
best_gbm_from_rand_grid.summary()

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,168.0,168.0,103534.0,7.0,7.0,7.0,13.0,82.0,43.809525


<br>

## Step 2: Build DRF Models using Random Grid Search and Extract the Best Model

In [13]:
# define the range of hyper-parameters for DRF grid search
# 27 combinations in total
hyper_params = {'sample_rate': [0.5, 0.6, 0.7],
                'col_sample_rate_per_tree': [0.7, 0.8, 0.9],
                'max_depth': [3, 5, 7]}

In [14]:
# Set up DRF grid search
# Add a seed for reproducibility
drf_rand_grid = H2OGridSearch(
                    H2ORandomForestEstimator(
                        model_id = 'drf_rand_grid', 
                        seed = 1234,
                        ntrees = 200,   
                        nfolds = 5,
                        fold_assignment = "Modulo",                 # needed for stacked ensembles
                        keep_cross_validation_predictions = True),  # needed for stacked ensembles
                    search_criteria = search_criteria, 
                    hyper_params = hyper_params)

In [15]:
# Use .train() to start the grid search
drf_rand_grid.train(x = features, 
                    y = 'quality', 
                    training_frame = wine_train)

drf Grid Build progress: |████████████████████████████████████████████████| 100%


In [16]:
# Sort and show the grid search results
drf_rand_grid_sorted = drf_rand_grid.get_grid(sort_by='mse', decreasing=False)
print(drf_rand_grid_sorted)

    col_sample_rate_per_tree max_depth sample_rate  \
0                        0.9         7         0.7   
1                        0.9         7         0.5   
2                        0.8         7         0.5   
3                        0.7         7         0.5   
4                        0.7         5         0.6   
5                        0.9         3         0.7   
6                        0.8         3         0.6   
7                        0.8         3         0.7   
8                        0.7         3         0.5   

                                                     model_ids  \
0  Grid_DRF_py_4_sid_85d6_model_python_1491487273261_5_model_5   
1  Grid_DRF_py_4_sid_85d6_model_python_1491487273261_5_model_6   
2  Grid_DRF_py_4_sid_85d6_model_python_1491487273261_5_model_4   
3  Grid_DRF_py_4_sid_85d6_model_python_1491487273261_5_model_1   
4  Grid_DRF_py_4_sid_85d6_model_python_1491487273261_5_model_0   
5  Grid_DRF_py_4_sid_85d6_model_python_1491487273261_5_model_2 

In [17]:
# Extract the best model from random grid search
best_drf_model_id = drf_rand_grid_sorted.model_ids[0]
best_drf_from_rand_grid = h2o.get_model(best_drf_model_id)
best_drf_from_rand_grid.summary()

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,200.0,200.0,239754.0,7.0,7.0,7.0,70.0,111.0,90.265


<br>

## Step 3: Build DNN Models using Random Grid Search and Extract the Best Model

In [18]:
# define the range of hyper-parameters for DNN grid search
# 81 combinations in total
hyper_params = {'activation': ['tanh', 'rectifier', 'maxout'],
                'hidden': [[50], [50,50], [50,50,50]],
                'l1': [0, 1e-3, 1e-5],
                'l2': [0, 1e-3, 1e-5]}

In [19]:
# Set up DNN grid search
# Add a seed for reproducibility
dnn_rand_grid = H2OGridSearch(
                    H2ODeepLearningEstimator(
                        model_id = 'dnn_rand_grid', 
                        seed = 1234,
                        epochs = 20,   
                        nfolds = 5,
                        fold_assignment = "Modulo",                # needed for stacked ensembles
                        keep_cross_validation_predictions = True), # needed for stacked ensembles
                    search_criteria = search_criteria, 
                    hyper_params = hyper_params)

In [20]:
# Use .train() to start the grid search
dnn_rand_grid.train(x = features, 
                    y = 'quality', 
                    training_frame = wine_train)

deeplearning Grid Build progress: |███████████████████████████████████████| 100%


In [21]:
# Sort and show the grid search results
dnn_rand_grid_sorted = dnn_rand_grid.get_grid(sort_by='mse', decreasing=False)
print(dnn_rand_grid_sorted)

    activation        hidden      l1      l2  \
0       Maxout      [50, 50]     0.0  1.0E-5   
1         Tanh  [50, 50, 50]     0.0  1.0E-5   
2         Tanh  [50, 50, 50]  1.0E-5  1.0E-5   
3       Maxout  [50, 50, 50]  1.0E-5  1.0E-5   
4       Maxout          [50]     0.0     0.0   
5         Tanh  [50, 50, 50]   0.001     0.0   
6       Maxout          [50]  1.0E-5   0.001   
7       Maxout          [50]   0.001     0.0   
8    Rectifier      [50, 50]  1.0E-5     0.0   

                                                              model_ids  \
0  Grid_DeepLearning_py_4_sid_85d6_model_python_1491487273261_6_model_8   
1  Grid_DeepLearning_py_4_sid_85d6_model_python_1491487273261_6_model_0   
2  Grid_DeepLearning_py_4_sid_85d6_model_python_1491487273261_6_model_7   
3  Grid_DeepLearning_py_4_sid_85d6_model_python_1491487273261_6_model_3   
4  Grid_DeepLearning_py_4_sid_85d6_model_python_1491487273261_6_model_5   
5  Grid_DeepLearning_py_4_sid_85d6_model_python_1491487273261_6_model

In [22]:
# Extract the best model from random grid search
best_dnn_model_id = dnn_rand_grid_sorted.model_ids[0]
best_dnn_from_rand_grid = h2o.get_model(best_dnn_model_id)
best_dnn_from_rand_grid.summary()

Status of Neuron Layers: predicting quality, regression, gaussian distribution, Quadratic loss, 6,351 weights/biases, 82.8 KB, 81,920 training samples, mini-batch size 1



,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
,1,11,Input,0.0,,,,,,,,,
,2,50,Maxout,0.0,0.0,1e-05,0.0017094,0.0008023,0.0,-0.0030162,0.1949095,0.3963432,0.0938767
,3,50,Maxout,0.0,0.0,1e-05,0.0366543,0.0406336,0.0,-0.0290520,0.1534271,0.9438809,0.0487866
,4,1,Linear,,0.0,1e-05,0.0005262,0.0001888,0.0,0.0140272,0.1109572,0.0195533,0.0000000


<br>

## Model Stacking

In [23]:
# Define a list of models to be stacked
# i.e. best model from each grid
all_ids = [best_gbm_model_id, best_drf_model_id, best_dnn_model_id]

In [24]:
# Set up Stacked Ensemble
ensemble = H2OStackedEnsembleEstimator(model_id = "my_ensemble",
                                       base_models = all_ids)

In [25]:
# use .train to start model stacking
# GLM as the default metalearner
ensemble.train(x = features, 
               y = 'quality', 
               training_frame = wine_train)

stackedensemble Model Build progress: |███████████████████████████████████| 100%


<br>

## Comparison of Model Performance on Test Data

In [26]:
print('Best GBM model from Grid (MSE) : ', best_gbm_from_rand_grid.model_performance(wine_test).mse())
print('Best DRF model from Grid (MSE) : ', best_drf_from_rand_grid.model_performance(wine_test).mse())
print('Best DNN model from Grid (MSE) : ', best_dnn_from_rand_grid.model_performance(wine_test).mse())
print('Stacked Ensembles        (MSE) : ', ensemble.model_performance(wine_test).mse())

Best GBM model from Grid (MSE) :  0.4013942890547201
Best DRF model from Grid (MSE) :  0.4781568285687009
Best DNN model from Grid (MSE) :  0.4995303971383784
Stacked Ensembles        (MSE) :  0.39692749423528306


<br>

<br>